# Compute distance between markup points

Import modules and define functions

In [7]:
import numpy as np
import pandas as pd
import os
import json

# numpy disable scientific notation for easier debugging
np.set_printoptions(suppress=True, precision=4)

# make prototype for storing CT, T1 and electrodes filenames
class SubjectFiles:
    def __init__(self, subject_root, ct_file, t1_file, electrodes_file):
        self.subject_root = subject_root
        self.ct_file = ct_file
        self.t1_file = t1_file
        self.electrodes_file = electrodes_file

    def ct_date(self):
        return self.get_date(self.ct_file)
    
    def t1_date(self):
        return self.get_date(self.t1_file)

    @staticmethod
    def get_date(filename):
        match = re.search(r"_(\d{8})_", filename)
        if match:
            return match.group(1)
        return None
    
def load_json(path: str) -> pd.DataFrame:
    markups_prediction_json = json.load(open(path, "r"))
    assert markups_prediction_json["markups"][0]["coordinateUnits"] == "mm"
    assert markups_prediction_json["markups"][0]["coordinateSystem"] == "LPS"
    df = pd.DataFrame(markups_prediction_json["markups"][0]["controlPoints"], columns=["label", "position"])
    df["position"] = df["position"].apply(lambda x: [-x[0], -x[1], x[2]]) # LPS -> RAS
    return df


Load data

In [8]:
input_dir = "../../Data"
subjects: list[SubjectFiles] = []

# walk through all subfolders and search for *CT*, *T1* and electrodes.fcsv files
for root, dirs, files in os.walk(input_dir):
    ct_path = None
    t1_path = None
    fcsv_path = None

    for file in files:
        if "CT" in file and file.endswith((".nii", ".nii.gz")):
            ct_path = file
        elif "T1" in file and not file.startswith("rand_affine_") and file.endswith((".nii", ".nii.gz")):
            t1_path = file
        elif file == "ContactDetector.mrk.json":
            fcsv_path = file
    if ct_path and t1_path and fcsv_path:
        root = root.replace(input_dir + os.sep, "")
        subjects.append(SubjectFiles(root, ct_path, t1_path, fcsv_path))

Compute distance between markup points:

In [9]:
markups_all = []
for subject in subjects:
    gt_df = pd.read_csv(os.path.join(input_dir, subject.subject_root, "electrodes.csv"), sep=";", names=["label", "r", "a", "s"])
    gt_df["position"] = gt_df[["r", "a", "s"]].apply(lambda x: list(x), axis=1)
    contact_detector_df = load_json(os.path.join(input_dir, subject.subject_root, "ContactDetector.mrk.json"))

    # merge gt and contact detector df on label
    markups = pd.merge(gt_df, contact_detector_df, on="label", how="outer", suffixes=("_gt", "_contact_detector"))
    markups["prefix"] = markups["label"].str.extract(r"(.*?)(\d+)$")[0]
    markups["contact"] = markups["label"].str.extract(r"(.*?)(\d+)$")[1]
    markups["subject_root"] = subject.subject_root
    # change column order
    markups = markups[["subject_root", "label", "prefix", "contact", "position_gt", "position_contact_detector"]]

    for index, row in markups.iterrows():
        markups.loc[index, "norm(gt,contact detector)"] = np.linalg.norm(np.array(row["position_gt"]) - np.array(row["position_contact_detector"]))

    markups_all.append(markups)

markups = pd.concat(markups_all)

# save to csv
markups.to_csv("results.csv", index=False)

markups

,subject_root,label,prefix,contact,position_gt,position_contact_detector,"norm(gt,contact detector)"
0,1738549,A1,A,1,"[-17.3922215945814, -8.16191350286358, -45.528...","[-17.492799236641616, -8.164179261111727, -45....",0.150626
1,1738549,A10,A,10,"[-47.0436357913221, -12.7798883262983, -35.823...","[-47.12243622458408, -12.801992321490616, -35....",0.086966
2,1738549,A11,A,11,"[-50.3099010562873, -13.3177426952358, -34.712...","[-50.35563611970673, -13.343252920285764, -34....",0.061803
3,1738549,A12,A,12,"[-53.5766813676251, -13.8579543909944, -33.598...","[-53.582587326651264, -13.882390359169676, -33...",0.039451
4,1738549,A13,A,13,"[-56.8437479243421, -14.399541981385, -32.4828...","[-56.90426748395015, -14.4332302464192, -32.47...",0.069870
...,...,...,...,...,...,...,...
192,890775,Xs5,Xs,5,"[-31.6970107352406, 15.3250862805842, 24.39156...","[-31.707537604390566, 15.321291129958169, 24.3...",0.058799
193,890775,Xs6,Xs,6,"[-31.7136500988301, 14.5341170126905, 27.80221...","[-31.730194193019244, 14.532472449668504, 27.7...",0.034323
194,890775,Xs7,Xs,7,"[-31.7375817707216, 13.7424540768876, 31.21256...","[-31.75524227706434, 13.743624113698715, 31.21...",0.017804
195,890775,Xs8,Xs,8,"[-31.7677936775634, 12.949672143704, 34.622531...","[-31.78085174821304, 12.975626719178038, 34.54...",0.080765


Table of missing contact detections:

In [10]:
markups[markups["position_contact_detector"].isna() | markups["position_gt"].isna()]

,subject_root,label,prefix,contact,position_gt,position_contact_detector,"norm(gt,contact detector)"


Group by subject_root and prefix, get max norm and mean norm for each group:

In [11]:
electrodes = markups.groupby(["subject_root", "prefix"]).agg({
    "subject_root": "first",
    "norm(gt,contact detector)": ["max", "mean"]
}).sort_values(("norm(gt,contact detector)", "max"), ascending=False)
electrodes

subject_root norm(gt,contact detector)          
                           first                       max      mean
subject_root prefix                                                 
1673284      H           1673284                  1.065802  0.350487
2116101      I           2116101                  0.619458  0.195625
2126753      F           2126753                  0.602017  0.183175
1923351      E           1923351                  0.501986  0.433557
2116101      F           2116101                  0.497942  0.112476
...                          ...                       ...       ...
1410956      D           1410956                  0.055277  0.040593
1812938      Ps          1812938                  0.053639  0.040347
1396542      Sd          1396542                  0.052164  0.031451
1460688      J           1460688                  0.051989  0.023118
1883612/i1   M        1883612/i1                  0.046058  0.027725

[1078 rows x 3 columns]

Consider mean norm > 1 as failed detection for particular electrode:

In [12]:
# for subject_root count norm.mean > 1
failed = electrodes[electrodes[("norm(gt,contact detector)", "mean")] > 1]

# save to csv
failed.to_csv("failed.csv", index=False)
failed

Empty DataFrame
Columns: [(subject_root, first), (norm(gt,contact detector), max), (norm(gt,contact detector), mean)]
Index: []